In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import pylab
import matplotlib.pyplot as plt
import scipy as sp
import glob

pylab.rcParams['figure.figsize'] = (22.0, 11.0)

In [9]:
#niwot = pd.read_pickle('/Users/barnhatb/Dropbox/CUB/ET magnitude and snowmelt/data/snotel/niwot_663/niwot.pcl')
niwot = pd.read_pickle('./data/niwot_snotel.pcl')
niwot.swe *= 2.54 # convert from inches to cm
niwot.swe *= 10. # convert from cm to mm

In [10]:
niwot.describe()

,swe,precip_accum,ATmax,ATmin,ATave,precip
count,12321.00000,12321.000000,9034.000000,9034.000000,9034.000000,12321.000000
mean,108.68318,15.432278,9.502558,-4.235635,1.834243,0.002224
std,128.52590,10.555857,9.993307,7.470492,7.996622,0.004892
min,0.00000,0.000000,-22.777778,-47.777778,-25.000000,0.000000
25%,0.00000,6.000000,1.666667,-9.444444,-3.888889,0.000000
50%,48.26000,14.500000,9.444444,-3.333333,1.666667,0.000000
75%,198.12000,24.500000,17.777778,1.111111,8.333333,0.002540
max,535.94000,41.700000,46.666667,13.888889,19.444444,0.076200


In [24]:
years = np.unique(niwot.index.year)[25:-3]
print 'Calendar Years: '+ str(years)
wyears = years[1:]
print 'Water Years: ' + str(wyears)

Calendar Years: [2005 2006 2007 2008 2009 2010 2011]
Water Years: [2006 2007 2008 2009 2010 2011]


In [25]:
cols = ['peakswe','peakswedate','snowfreedate','ablationrate','ablationseason', 'growseasonlength']
SWE = pd.DataFrame(np.zeros([len(wyears),len(cols)]), index=wyears, columns=cols)

In [26]:
SWE

,peakswe,peakswedate,snowfreedate,ablationrate,ablationseason,growseasonlength
2006,0,0,0,0,0,0
2007,0,0,0,0,0,0
2008,0,0,0,0,0,0
2009,0,0,0,0,0,0
2010,0,0,0,0,0,0
2011,0,0,0,0,0,0


In [27]:
for year in wyears:
    SWE.loc[year,'peakswe'] = niwot.loc[
str(year-1)+'-10-01':str(year)+'-09-30'].swe.max() # extract the peak swe for the water year selected
    
    SWE.loc[year,'peakswedate'] = niwot.loc[
str(year-1)+'-10-01':str(year)+'-09-30'].swe.argmax() # extract the date of peak swe
    
    SWE.loc[year,'snowfreedate'] = niwot.loc[
str(year)+'-01-01':str(year)+'-09-30'].swe.argmin() # extract the first snow free date
    
    timedelta = SWE.loc[year,'snowfreedate'] - SWE.loc[year,'peakswedate']

    SWE.loc[year,'ablationrate'] = SWE.loc[year,'peakswe']/timedelta.days # [mm/day]
    SWE.loc[year, 'ablationseason'] = int(timedelta.days)
    
    timedelta = pd.datetime.strptime(str(year)+'-09-30','%Y-%m-%d') - SWE.loc[year,'peakswedate']
    
    SWE.loc[year,'growseasonlength'] = timedelta.days

In [28]:
SWE

,peakswe,peakswedate,snowfreedate,ablationrate,ablationseason,growseasonlength
2006,325.12,2006-03-27,2006-05-17,6.374902,51,187
2007,454.66,2007-04-27,2007-06-03,12.288108,37,156
2008,378.46,2008-04-18,2008-06-02,8.410222,45,165
2009,406.40,2009-04-19,2009-05-25,11.288889,36,164
2010,317.50,2010-04-08,2010-06-02,5.772727,55,175
2011,431.80,2011-05-22,2011-06-13,19.627273,22,131


In [29]:
#SWE.to_pickle('/Users/barnhatb/Theo/Volumes/Users/Theo/greenness_timing/niwot_snowpack_metrics.pcl')
SWE.to_pickle('./data/niwot_snowpack_metrics.pcl')